In [1]:
# importing prerequisites
import sys
import requests
import tarfile
import json
import numpy as np
from os import path
from PIL import Image
from PIL import ImageFont, ImageDraw
from glob import glob
from matplotlib import pyplot as plt
import pandas as pd
%matplotlib inline


import random
import pprint
import time
from optparse import OptionParser
import pickle
import math
import cv2
import copy
import tensorflow as tf
import os


In [2]:
fname = 'examples.tar.gz'
url = 'https://dax-cdn.cdn.appdomain.cloud/dax-publaynet/1.0.0/' + fname
r = requests.get(url)
open(fname , 'wb').write(r.content)

6461141

In [3]:
# Extracting the dataset
tar = tarfile.open(fname)
tar.extractall()
tar.close()

In [4]:
# Verifying the file was extracted properly
data_path = "examples/"
path.exists(data_path)

True

In [5]:
# Define color code
colors = {'title': (255, 0, 0),
          'text': (0, 255, 0),
          'figure': (0, 0, 255),
          'table': (255, 255, 0),
          'list': (0, 255, 255)}

In [6]:
# Function to viz the annotation
def markup(image, annotations):
    ''' Draws the segmentation, bounding box, and label of each annotation
    '''
    draw = ImageDraw.Draw(image, 'RGBA')
    for annotation in annotations:
        # Draw segmentation
        draw.polygon(annotation['segmentation'][0],
                     fill=colors[samples['categories'][annotation['category_id'] - 1]['name']] + (64,))
        # Draw bbox
        draw.rectangle(
            (annotation['bbox'][0],
             annotation['bbox'][1],
             annotation['bbox'][0] + annotation['bbox'][2],
             annotation['bbox'][1] + annotation['bbox'][3]),
            outline=colors[samples['categories'][annotation['category_id'] - 1]['name']] + (255,),
            width=2
        )
        # Draw label
        w, h = draw.textsize(text=samples['categories'][annotation['category_id'] - 1]['name'],
                             font=font)
        if annotation['bbox'][3] < h:
            draw.rectangle(
                (annotation['bbox'][0] + annotation['bbox'][2],
                 annotation['bbox'][1],
                 annotation['bbox'][0] + annotation['bbox'][2] + w,
                 annotation['bbox'][1] + h),
                fill=(64, 64, 64, 255)
            )
            draw.text(
                (annotation['bbox'][0] + annotation['bbox'][2],
                 annotation['bbox'][1]),
                text=samples['categories'][annotation['category_id'] - 1]['name'],
                fill=(255, 255, 255, 255),
                font=font
            )
        else:
            draw.rectangle(
                (annotation['bbox'][0],
                 annotation['bbox'][1],
                 annotation['bbox'][0] + w,
                 annotation['bbox'][1] + h),
                fill=(64, 64, 64, 255)
            )
            draw.text(
                (annotation['bbox'][0],
                 annotation['bbox'][1]),
                text=samples['categories'][annotation['category_id'] - 1]['name'],
                fill=(255, 255, 255, 255),
                font=font
            )
    return np.array(image)

(Filename,XMin,XMax,Ymin, YMax, ClassLabel)

Change file name as you need for JSON file containing annotation details.

In [7]:
# Parse the JSON file and read all the images and labels
with open('examples/samples.json', 'r') as fp:
    samples = json.load(fp)
# Index images
images = {}
for image in samples['images']:
    images[image['id']] = {'file_name': "examples/" + image['file_name'], 'annotations': []}
for ann in samples['annotations']:
    images[ann['image_id']]['annotations'].append(ann)

In [8]:

train_df_dict = {
    "file_name": [],
    "x_min": [],
    "y_min": [],
    "x_max": [],
    "y_max": [],
    "class_name": []
}
for i, (_, image) in enumerate(images.items()):

    for ann in image['annotations']:
        train_df_dict['file_name'].append(image['file_name'])
        train_df_dict['x_min'].append(ann['bbox'][0])
        train_df_dict['x_max'].append(ann['bbox'][0] + ann['bbox'][2])
        train_df_dict['y_min'].append(ann['bbox'][1])
        train_df_dict['y_max'].append(ann['bbox'][1] +  ann['bbox'][3])
        train_df_dict['class_name'].append(samples['categories'][ann['category_id'] - 1]['name'])
    

In [9]:
# df (file_name,x_min, x_max, y_min, y_max, class_name)
train_df = pd.DataFrame.from_dict(data=train_df_dict)


In [10]:
train_df

,file_name,x_min,y_min,x_max,y_max,class_name
0,examples/PMC5491943_00004.jpg,121.89,41.80,549.88,76.30,text
1,examples/PMC5491943_00004.jpg,121.89,83.47,549.83,129.97,text
2,examples/PMC5491943_00004.jpg,121.89,163.31,549.97,247.00,text
3,examples/PMC5491943_00004.jpg,121.89,254.37,549.85,362.46,text
4,examples/PMC5491943_00004.jpg,121.89,369.82,336.51,380.33,text
...,...,...,...,...,...,...
188,examples/PMC3654277_00006.jpg,308.61,696.04,534.00,709.48,text
189,examples/PMC3654277_00006.jpg,50.58,276.69,548.74,309.71,text
190,examples/PMC3654277_00006.jpg,50.83,70.68,546.24,271.09,figure
191,examples/PMC3654277_00006.jpg,325.94,718.30,548.77,743.11,list


In [11]:
#image = plt.imread(train_df.iloc[0].file_name)

In [12]:
#type(int(train_df['y_min'][1]))

int

In [13]:
train_df.to_csv('annotate.txt', header=None, index=None, sep=',')